In [ ]:
#| default_exp quantize.quantize_callback

In [ ]:
#| include: false
import warnings
warnings.filterwarnings('ignore')
from nbdev.showdoc import *
from fastai.vision.all import *

In [ ]:
#| export
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from fastai.vision.all import *
from fastai.callback.all import *

from torch.ao.quantization import get_default_qat_qconfig_mapping
import torch.ao.quantization.quantize_fx as quantize_fx
from torch.ao.quantization.quantize_fx import convert_fx, prepare_fx

In [ ]:
#| include: false
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

In [ ]:
#| export
class QuantizeCallback(Callback):
    def __init__(self, qconfig_mapping=None, backend='x86'):
        self.qconfig_mapping = qconfig_mapping or get_default_qat_qconfig_mapping(backend)

    def before_fit(self):
        example_inputs, _ = next(iter(self.dls.train))
        self.learn.model = quantize_fx.prepare_qat_fx(self.learn.model, self.qconfig_mapping, example_inputs)

    def after_fit(self):
        self.learn.model.eval()
        self.learn.model = quantize_fx.convert_fx(self.learn.model.to('cpu'))

In [ ]:
show_doc(QuantizeCallback)

---

### QuantizeCallback

>      QuantizeCallback (qconfig_mapping=None, backend='x86')

Basic class handling tweaks of the training loop by changing a `Learner` in various events

In [ ]:
import timm
pretrained_resnet_34 = timm.create_model('resnet34', pretrained=True)
learn = Learner(dls, pretrained_resnet_34, metrics=accuracy)
learn.model.fc = nn.Linear(512, 2)
learn.fit_one_cycle(10, cbs=QuantizeCallback())

epoch,train_loss,valid_loss,accuracy,time
0,0.593340,0.501450,0.718539,00:12
1,0.395149,0.378658,0.830853,00:11
2,0.247482,0.231792,0.901218,00:11
3,0.164855,0.265116,0.892422,00:11
4,0.112191,0.228632,0.914073,00:11
5,0.070206,0.214058,0.929635,00:11
6,0.050446,0.202638,0.937754,00:11
7,0.033936,0.203362,0.941137,00:11
8,0.024784,0.201417,0.938430,00:11
9,0.022369,0.193307,0.941813,00:12


In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)
learn.unfreeze()
learn.fit(5, cbs=QuantizeCallback())

epoch,train_loss,valid_loss,accuracy,time
0,0.571546,0.424050,0.789581,00:06
1,0.469406,0.363851,0.843708,00:06
2,0.407703,0.399239,0.817997,00:06
3,0.375065,0.309377,0.865359,00:06
4,0.323774,0.331475,0.873478,00:06


In [ ]:
learn.model

GraphModule(
  (0): Module(
    (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.029317768290638924, zero_point=0, padding=(3, 3))
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Module(
      (0): Module(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.017887497320771217, zero_point=0, padding=(1, 1))
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.0466480627655983, zero_point=66, padding=(1, 1))
      )
      (1): Module(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.017889995127916336, zero_point=0, padding=(1, 1))
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.07470479607582092, zero_point=66, padding=(1, 1))
      )
    )
    (5): Module(
      (0): Module(
        (conv1): QuantizedConvReLU2d(64, 128, kernel_size=(3, 3), stride=(2, 2), scale=0.01743861660361